In [84]:
import numpy as np
import optuna
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [85]:
# 二分类问题
data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
print(data.shape)

(569, 30)


In [86]:
# 第一个参数必须为trial
def objective(trial, data, target):
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.Callback for XGBoost to prune unpromising trials.
    # 推荐使用此方式
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial,
                                                                 # An evaluation metric for pruning
                                                                 # 可选值为:"validation" + "-" + xgboost模型的"eval_metric"参数的值
                                                                 # 如validation-error/validation-acu/validaton-merror/validation-auc
                                                                 observation_key="validation-logloss")
    # 也可以在次进行交叉验证
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.around(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [87]:

study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=15), direction="maximize")
func = lambda trial: objective(trial, data, target)
study.optimize(func, n_trials=100)

[I 2021-09-18 18:21:37,989] A new study created in memory with name: no-name-77befefd-bf71-48b1-9ab1-84cf182c0911


[0]	validation-logloss:0.63164
[1]	validation-logloss:0.58505
[2]	validation-logloss:0.51620
[3]	validation-logloss:0.48427
[4]	validation-logloss:0.44618
[5]	validation-logloss:0.40822
[6]	validation-logloss:0.38995
[7]	validation-logloss:0.36997
[8]	validation-logloss:0.35431
[9]	validation-logloss:0.34221


[I 2021-09-18 18:21:38,004] Trial 0 finished with value: 0.9300699300699301 and parameters: {'booster': 'gblinear', 'lambda': 0.10079739033893817, 'alpha': 0.5882764307068763}. Best is trial 0 with value: 0.9300699300699301.


[0]	validation-logloss:0.44012
[1]	validation-logloss:0.36369
[2]	validation-logloss:0.30545
[3]	validation-logloss:0.27868
[4]	validation-logloss:0.26998
[5]	validation-logloss:0.23068
[6]	validation-logloss:0.21554
[7]	validation-logloss:0.20514
[8]	validation-logloss:0.19562
[9]	validation-logloss:0.17078


[I 2021-09-18 18:21:38,016] Trial 1 finished with value: 0.951048951048951 and parameters: {'booster': 'gblinear', 'lambda': 1.5474373727693423e-08, 'alpha': 3.903258473541346e-06}. Best is trial 1 with value: 0.951048951048951.


...

[I 2021-09-18 18:21:41,983] Trial 97 finished with value: 0.965034965034965 and parameters: {'booster': 'gbtree', 'lambda': 0.0026510128601349037, 'alpha': 1.8247856780924252e-08, 'max_depth': 4, 'eta': 0.3733493549875287, 'gamma': 0.21602273942389183, 'grow_policy': 'depthwise'}. Best is trial 84 with value: 0.986013986013986.


[0]	validation-logloss:0.62397
[1]	validation-logloss:0.57022
[2]	validation-logloss:0.52441
[3]	validation-logloss:0.48758
[4]	validation-logloss:0.45455
[5]	validation-logloss:0.42722
[6]	validation-logloss:0.40397
[7]	validation-logloss:0.37949
[8]	validation-logloss:0.35975
[9]	validation-logloss:0.34019


[I 2021-09-18 18:21:42,022] Trial 98 finished with value: 0.9020979020979021 and parameters: {'booster': 'gbtree', 'lambda': 0.005451595609699619, 'alpha': 3.0209913684559164e-07, 'max_depth': 3, 'eta': 0.09208796940830002, 'gamma': 0.11909984071344731, 'grow_policy': 'depthwise'}. Best is trial 84 with value: 0.986013986013986.


[0]	validation-logloss:0.43411
[1]	validation-logloss:0.30072
[2]	validation-logloss:0.22817
[3]	validation-logloss:0.18011
[4]	validation-logloss:0.15090
[5]	validation-logloss:0.13846
[6]	validation-logloss:0.12341
[7]	validation-logloss:0.11885
[8]	validation-logloss:0.10647
[9]	validation-logloss:0.10140


[I 2021-09-18 18:21:42,065] Trial 99 finished with value: 0.972027972027972 and parameters: {'booster': 'gbtree', 'lambda': 0.9894992824882964, 'alpha': 1.9023450176696273e-08, 'max_depth': 4, 'eta': 0.39455152368203, 'gamma': 0.027002706941291962, 'grow_policy': 'depthwise'}. Best is trial 84 with value: 0.986013986013986.


In [88]:
# 最优结果
print(study.best_value)

# 最优参数
print(study.best_params)

0.986013986013986
{'booster': 'gbtree', 'lambda': 0.30156068666026076, 'alpha': 0.0012420140902708475, 'max_depth': 5, 'eta': 0.5669195983130255, 'gamma': 0.08197222350324723, 'grow_policy': 'depthwise'}
